In [1]:
%%capture
!pip install datasets
!pip install transformers
!pip install --upgrade accelerate
!pip install evaluate

In [4]:
from datasets import load_dataset
from huggingface_hub import list_datasets
print([dataset.id for dataset in list_datasets()])

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [2]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModelForTokenClassification
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling
import datasets
from evaluate import load
from evaluate import evaluator
import evaluate
import numpy as np
import torch
import copy

from collections import defaultdict

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device_id = 0 if str(device) == 'cuda' else -1

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [5]:
bert_model = pipeline('fill-mask', model='bert-base-cased', device=device_id)
print(bert_model("I went to an [MASK] restaurant and ordered pasta."))



config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Device set to use cpu


[{'score': 0.8473349213600159, 'token': 2169, 'token_str': 'Italian', 'sequence': 'I went to an Italian restaurant and ordered pasta.'}, {'score': 0.03583117946982384, 'token': 1890, 'token_str': 'Indian', 'sequence': 'I went to an Indian restaurant and ordered pasta.'}, {'score': 0.016542520374059677, 'token': 1385, 'token_str': 'old', 'sequence': 'I went to an old restaurant and ordered pasta.'}, {'score': 0.006617061328142881, 'token': 3427, 'token_str': 'empty', 'sequence': 'I went to an empty restaurant and ordered pasta.'}, {'score': 0.005878610536456108, 'token': 6210, 'token_str': 'Egyptian', 'sequence': 'I went to an Egyptian restaurant and ordered pasta.'}]


In [8]:
#zero shot bert model
bert_model = pipeline('fill-mask', model='bert-base-cased', device=device_id)
print(bert_model("I went to an [MASK] restaurant and ordered pasta."))
bert_model("I just saw a movie today and it was really great. My opinion of the movie is [MASK].", targets=["positive", "negative"])

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


[{'score': 0.8473349213600159, 'token': 2169, 'token_str': 'Italian', 'sequence': 'I went to an Italian restaurant and ordered pasta.'}, {'score': 0.03583117946982384, 'token': 1890, 'token_str': 'Indian', 'sequence': 'I went to an Indian restaurant and ordered pasta.'}, {'score': 0.016542520374059677, 'token': 1385, 'token_str': 'old', 'sequence': 'I went to an old restaurant and ordered pasta.'}, {'score': 0.006617061328142881, 'token': 3427, 'token_str': 'empty', 'sequence': 'I went to an empty restaurant and ordered pasta.'}, {'score': 0.005878610536456108, 'token': 6210, 'token_str': 'Egyptian', 'sequence': 'I went to an Egyptian restaurant and ordered pasta.'}]


[{'score': 0.027820035815238953,
  'token': 3112,
  'token_str': 'positive',
  'sequence': 'I just saw a movie today and it was really great. My opinion of the movie is positive.'},
 {'score': 0.01648258976638317,
  'token': 4366,
  'token_str': 'negative',
  'sequence': 'I just saw a movie today and it was really great. My opinion of the movie is negative.'}]

In [9]:
dataset = load_dataset("yelp_review_full")

README.md:   0%|          | 0.00/6.72k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/299M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/23.5M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/650000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [11]:
def concatenate_mask(row):
    template = " I give it a score of [MASK] out of 5."
    row["input"] = row["text"] + template
    return row

test_data = dataset['test'].map(concatenate_mask)
test_data = dataset['test'].map(concatenate_mask)

In [27]:
# sampled_data = data.shuffle(seed=42).select(range(100))
print(bert_model("I am italian! My favorite dessert is [MASK]."))
print(bert_model("I am currently speaking in the [MASK] language.")[0])

[{'score': 0.059903744608163834, 'token': 6762, 'token_str': 'yours', 'sequence': 'I am italian! My favorite dessert is yours.'}, {'score': 0.04709651321172714, 'token': 8888, 'token_str': 'chocolate', 'sequence': 'I am italian! My favorite dessert is chocolate.'}, {'score': 0.03440161421895027, 'token': 1303, 'token_str': 'here', 'sequence': 'I am italian! My favorite dessert is here.'}, {'score': 0.0317012295126915, 'token': 20392, 'token_str': 'dessert', 'sequence': 'I am italian! My favorite dessert is dessert.'}, {'score': 0.0229936596006155, 'token': 1142, 'token_str': 'this', 'sequence': 'I am italian! My favorite dessert is this.'}]
{'score': 0.18785078823566437, 'token': 1483, 'token_str': 'English', 'sequence': 'I am currently speaking in the English language.'}


AutoModelForMaskedLM - Language modeling Head <br>
AutoModelForSequenceClassification - Classification Head <br>
AutoModelForTokenClassification - Token Classification Head



Classification Head for n=5 labels

In [ ]:
# finetune classification model on downstream task for n=5 labels
classification_head_bert = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

In [13]:
dataset = load_dataset("yelp_review_full")
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
def tokenize_function(row):
  row = tokenizer(row["text"], padding="max_length", truncation=True, max_length=512)

  return row

train_data = dataset["train"].shuffle(seed=42).select(range(1000)).map(tokenize_function)
eval_data = dataset["test"].shuffle(seed=42).select(range(1000)).map(tokenize_function)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [39]:
def compute_metrics(eval_pred):
    probabilities, labels = eval_pred
    max_indices = np.argmax(np.array(probabilities),axis=-1)
    acc = np.sum(max_indices == labels)/len(labels)
    return {'accuracy': acc}

probabilities = np.array([[0.1, 0.7, 0.2],
                          [0.7, 0.4, 0.3],
                          [0.8, 0.1, 0.1],
                          [0.8, 0.1, 0.1]])
labels = np.array([1, 1, 0,1])
compute_metrics((probabilities,labels))

metric = load("accuracy")
results = metric.compute(predictions=[0, 1, 0, 1], references=[0, 1, 1, 1])
print(results)

{'accuracy': 0.75}


In [40]:
# Step 3: Specify the TrainingArguments and Initialize the Trainer (Do not change!)
training_args = TrainingArguments(
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    output_dir="yelp-training",
    learning_rate=5e-05,
    num_train_epochs=3.0,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    full_determinism=True,
    report_to="none"
)

trainer = Trainer(
    model=classification_head_bert,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=eval_data,
    compute_metrics=compute_metrics
)

/Users/juliasusser/myenv/lib/python3.12/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [41]:
trainer.train()

KeyboardInterrupt: 

In [ ]:
trainer.save_model('./yelpBERT')

In [ ]:
yelpBERT = pipeline('text-classification', model='./yelpBERT', tokenizer=tokenizer, device=device_id)
yelpBERT(["This place was amazing!",
          "This place was good",
          "This place was fine, there were good and bad parts.",
          "This place was pretty bad",
          "This place was amazing"])

In [ ]:
task_evaluator = evaluator("text-classification")

eval_results = task_evaluator.compute(
    model_or_pipeline=yelpBERT,
    data=eval_data,
    metric=evaluate.load("accuracy"),
    label_mapping={"LABEL_0": 0, "LABEL_1": 1, "LABEL_2": 2, "LABEL_3": 3, "LABEL_4": 4}
)

print(eval_results)

yelpBERT = pipeline('text-classification', model='./yelpBERT', tokenizer=tokenizer, device=device_id)

In [ ]:
#use same classification head for natural language inference
#SEP token to separate parts of the input


mnli = load_dataset("multi_nli")

def is_labeled(row):
  if row['label'] == -1: return False
  return True

mnli = mnli.filter(is_labeled)

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

def tokenize_function(row):
  row = tokenizer(row["text"], padding="max_length", truncation=True, max_length=512)
  return row

train_mnli = mnli["train"].shuffle(seed=42).select(range(1000)).map(tokenize_function)
eval_mnli = mnli["validation_matched"].shuffle(seed=42).select(range(1000)).map(tokenize_function)

README.md:   0%|          | 0.00/8.89k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/214M [00:00<?, ?B/s]

In [ ]:
training_args = TrainingArguments(
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    output_dir="mnli-training",
    learning_rate=5e-05,
    num_train_epochs=3.0,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    full_determinism=True
)

trainer = Trainer(
    model=classification_head_bert,
    args=training_args,
    train_dataset=train_mnli,
    eval_dataset=eval_mnli,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()
trainer.save_model('./mnliBERT')

In [ ]:
mnliBERT = pipeline('text-classification', model='./mnliBERT', tokenizer=tokenizer, device=device_id)

In [ ]:
task_evaluator = evaluator("text-classification")

eval_results = task_evaluator.compute(
    model_or_pipeline=mnliBERT,
    data=eval_mnli,
    input_column="concat",
    label_column="label",
    metric=evaluate.load("accuracy"),
    label_mapping={"LABEL_0": 0, "LABEL_1": 1, "LABEL_2": 2}
)

print(eval_results)